#  Project description
You've been offered an internship in the analytical department at Yandex.Afisha. Your first task is to help optimize marketing expenses.

## Description of the data
The **visits** table (server logs with data on website visits):
* Uid — user's unique identifier
* Device — user's device
* Start Ts — session start date and time
* End Ts — session end date and time
* Source Id — identifier of the ad source the user came from.

All dates in this table are in YYYY-MM-DD format.

The **orders** table (data on orders):
* Uid — unique identifier of the user making an order
* Buy Ts — order date and time
* Revenue — Yandex.Afisha's revenue from the order

The **costs** table (data on marketing expenses):
* source_id — ad source identifier
* dt — date
* costs — expenses on this ad source on this day

## Initialization


In [1]:
%pip install plotly

Note: you may need to restart the kernel to use updated packages.


In [2]:
%pip install sidetable

Note: you may need to restart the kernel to use updated packages.


In [3]:
# Loading all the libraries
import pandas as pd
import numpy as np
from datetime import datetime
import matplotlib.pyplot as plt
import seaborn as sns
import sidetable
import warnings
import plotly.express as px
import plotly.graph_objects as go

# Step 1. Download the data and prepare it for analysis
Store the data on visits, orders, and expenses in variables. Optimize the data for analysis. Make sure each column contains the correct data type.

In [4]:
try:
    visits = pd.read_csv('visits_log_us.csv', dtype = {'Device':'category'}, parse_dates=['Start Ts', 'End Ts'])
except:
    visits = pd.read_csv('/datasets/visits_log_us.csv', dtype = {'Device':'category'}, parse_dates=['Start Ts', 'End Ts'])
try:
    orders = pd.read_csv('orders_log_us.csv', parse_dates=['Buy Ts'])
except:
    orders = pd.read_csv('/datasets/orders_log_us.csv',parse_dates=['Buy Ts'])  
try:
    costs = pd.read_csv('costs_us.csv',parse_dates=['dt'])
except:
    costs = pd.read_csv('/datasets/costs_us.csv',parse_dates=['dt'])  
    
    

FileNotFoundError: [Errno 2] No such file or directory: '/datasets/visits_log_us.csv'

In [ ]:
visits.head()

In [ ]:
visits.info()

In [ ]:
visits.stb.missing(style=True)

In [ ]:
visits.duplicated().sum()

The table contains 359400 rows 5 columns of non-missing values. All datatypes are correct, no duplicates found. Let's change column names to lowercase and replace spaces with underscores.

In [ ]:
visits.columns=visits.columns.str.lower()
visits.columns=visits.columns.str.replace(' ', '_')

In [ ]:
visits.head()

Now **visits** table is ready for the analysis. Let's check **orders** dataset.

In [ ]:
orders.head()


In [ ]:
orders.info()

In [ ]:
orders.stb.missing(style=True)

Everything is ok except column names. Let's change them to lowercase and replace spaces with underscores.


In [ ]:
orders.columns=orders.columns.str.lower()
orders.columns=orders.columns.str.replace(' ', '_')

In [ ]:
orders.head()

In [ ]:
orders.duplicated().sum()

In [ ]:
orders.query('revenue == 0').revenue.value_counts()

Now **orders** table is ready too. Let's check **costs** dataset.

In [ ]:
costs.head()

In [ ]:
costs.info()

In [ ]:
costs.duplicated().sum()

The smallest dataset contains 2542 rows 3 columns of non-missing values, no duplicates. All datatypes and column names don't need changing. *costs* table is ready for the analysis.

The 3 datasets are interrelated: user_ids in visits are the same as user_ids in orders, source_ids in visits are the same  as source_ids in costs, they serve as foreign keys.

In [ ]:
visits['start_ts']

Let's add columns with date, week, month values for visits and orders, they will be necessary for cohort analysis.

In [ ]:
visits['date']=visits['start_ts'].dt.to_period('D').dt.to_timestamp()
visits['week'] = visits['start_ts'].dt.to_period('W').dt.to_timestamp()
visits['month'] = visits['start_ts'].dt.to_period('M').dt.to_timestamp()
orders['date']=orders['buy_ts'].dt.to_period('D').dt.to_timestamp()
orders['week'] = orders['buy_ts'].dt.to_period('W').dt.to_timestamp()
orders['month'] = orders['buy_ts'].dt.to_period('M').dt.to_timestamp()
costs['month'] = costs['dt'].dt.to_period('M').dt.to_timestamp()

In [ ]:
visits.head()

In [ ]:
#check datatype of new columns
visits.info()

In [ ]:
orders.head()

In [ ]:
costs.head()

**Conclusion**. Three dataframes were downloaded, no missing values and duplicates found, all datatypes are correct. Column names were changed to lowercase, spaces were replaced with underscores for consistency. Columns containing dates, weeks, months for were added, they are in datetime64[ns] format. Now we have **visits**, **orders** and **costs** datasets ready for analysis.

# Step 2. Make reports and calculate metrics:

## Product
### How many people use it every day, week, and month?


Let's find period of time the data is collected over:

In [ ]:
visits.start_ts.describe(datetime_is_numeric=True)

So the data is stored over the year from June 2017 till May 2018. If we group data by date, week, month and count unique users per period, it'll give us **DAU, WAU** and **MAU** metrics.

#### DAU

In [ ]:
dau=visits.groupby(visits['date']).uid.nunique().reset_index()
dau.head()

In [ ]:
dau.describe()

So DAU varies from 1 to 3319 users per day, with mean value around 908(median is a little greater - 921). Let's plot line graph for DAU over the year.

In [ ]:
fig = px.line(dau,x="date", y="uid",title='DAU')
#adding reference line with average DAU over time
fig.add_hline(y=dau['uid'].mean(),line_dash="dash", line_color="purple", annotation_text="average DAU",
             annotation_position="top left")
fig.show()

For nearly half a year(October - March) DAU is above average with the peak value  on Nov.24 (BLACK FRIDAY?), before the winter holidays(Cristmass, New Year). There's also a peak on May 31(end of the scool year?), a rise on March 25 then drop down to min values on March 31(maybe after spring holidays or some technical issues with the app?). By  July 17 the line suddenly rises and falls -  maybe there was some add compain or another factor influensed it.  We can also see small regular rises and falls during the week. Let's check if number  of users varys with day of the week.

In [ ]:
visits['dow'] = visits['date'].dt.dayofweek
visits.dow

In [ ]:
dow_visits = visits.groupby(['dow'])['uid'].count().reset_index()
dow_visits

In [ ]:
dow_visits.plot(kind='bar', x='dow', y='uid', color='blue')
plt.xlabel('Day of Week')
plt.ylabel('number of visits')
plt.title('Number of visits:  Day of Week')
plt.show()

So, people open the app most often on Wednesday, then on Sunday, while min number of visits is on Saturday and Friday. These fluctuations form certain week cycle.

#### WAU.
Now let's turn to weekly visits and WAU.

In [ ]:
wau=visits.groupby(visits['week']).uid.nunique().reset_index()
wau.head()

In [ ]:
wau.describe()

WAU varies from 2021 to 10586 users per week, the mean is 5716 (the median is 5740 - not much greater). Let's plot line graph for WAU

In [ ]:
import plotly.graph_objects as go
fig = go.Figure(layout=go.Layout(
        title=go.layout.Title(text="WAU")))
fig.add_trace(go.Scatter(x=wau['week'], 
                        y=wau['uid'],
                    mode='lines+markers',
                    name='lines+markers',line = dict(color='purple', width=5)))
#adding reference line with average DAU over time
fig.add_hline(y=wau['uid'].mean(),line_dash="dash", line_color="red", annotation_text="average WAU",
             annotation_position="top left")
fig.show()

Again we see max WAU in the end of november, minor peaks on October 2, at the end of January 29, March 19. min values are in July-August weeks. As a whole WAU is below average from the end of March till the middle of September.

#### MAU

In [ ]:
mau=visits.groupby(visits['month']).uid.nunique().reset_index()
mau

In [ ]:
mau.describe()

MAU varies from 116331 to 32797 users per month, the mean is 23228. 

In [ ]:
plot=sns.barplot(data=mau, x="month", y="uid")
plot.set(title="MAU")
plot.tick_params(axis='x', rotation=90)

Again we see MAU above average in October - March with max values in November and December. Since Aprile MAW is below average, in summer people buy less, min MAU value is in August.

### How many sessions are there per day? per user?


Since one user might have more than one session, let's calculate **number of sessions per day** by counting all users (not just unique) per day.

In [ ]:
sessions_day=visits.groupby(['date'])['uid'].count().reset_index()
sessions_day.head()

In [ ]:
sessions_day.describe()

Number of sessions per day varies from 1 to 4042, the mean is 987 (the median is 1003 -  there's some right skew).

In [ ]:
fig = px.line(sessions_day,x="date", y="uid",title='Number of Sessions per Day')
#adding reference line with average DAU over time
fig.add_hline(y=sessions_day['uid'].mean(),line_dash="dash", line_color="purple", annotation_text="average sessions_day",
             annotation_position="top left")
fig.show()

Let's find average **number of sessions per user** by dividing total number of sessions per day by DAU and explore the dynamics.

In [ ]:
day_dynamics=pd.merge(dau, sessions_day, on='date', how='inner')
day_dynamics.head()

In [ ]:
day_dynamics.columns =['date', 'dau', 'sessions_day']
day_dynamics.head()

In [ ]:
day_dynamics['sessions_user'] = day_dynamics['sessions_day']/ day_dynamics['dau']
day_dynamics.head()

In [ ]:
day_dynamics.sessions_user.describe()

So, number of sessions per user varies from 1 to 1.22 with the mean 1.08. Let's plot line graph.

In [ ]:
#let's plot sessions per user
fig = px.line(day_dynamics,
              x="date", y="sessions_user",title='Sessions per user')
fig.add_hline(y=day_dynamics['sessions_user'].mean(),line_dash="dash", line_color="purple", annotation_text="average sessions_user",
             annotation_position="top left")         
fig.show()

**Number of sessions per user** is very stable, normally distributed (with mean = median = 1.08 and std  only 0.02)
There are regular weekly cycles of ups and downs. Max value is again on Nov 24(Black Friday), smaller peaks on Feb 1, Mar 26, May 31, Jun 7, Jul 17, min on Mar 31.

Although number of sessions per day is greater than DAU, the overall picture of distribution  of these metrics with  peaks and week cycles is pretty similar. If we plot them on the same graph it will be more evident.

In [ ]:
fig = go.Figure()
fig.add_trace(go.Scatter(x=day_dynamics['date'], 
                        y=day_dynamics['dau'],
                    mode='lines',
                    name='DAU',line = dict(color='yellow', width=2 )))

fig.add_trace(go.Scatter(x=day_dynamics['date'], 
                        y=day_dynamics['sessions_day'], 
                       
                    mode='lines',
                    name='sessions',line = dict(color='black', width=2 )))


fig.update_layout(
    showlegend=True,
    plot_bgcolor="grey",
    margin=dict(t=10,l=10,b=10,r=10)
)


fig.show()

So, the line for number of sessions per day almost repeats the line for DAU a little higher.

### What is the length of each session?


We have start_ts and end_ts for each session, so **session length** will be:

In [ ]:
visits['session_length']= visits['end_ts']-visits['start_ts']
visits.session_length.head()

In [ ]:
visits.session_length.describe()

There's min session_length value -1 day + 23:14:00, which is absolutely impossible. Let's find the row in visits table:

In [ ]:
visits.query('session_length =="-1 days +23:14:00"')

Here start_ts is greater than end_ts and give negative session_length - that's an error! What if there is more than 1 such case?

In [ ]:
visits.query('start_ts > end_ts')

There're only 2 cases, both on the same day almost the same time(at night!), so maybe there were problems with internet connection or some other technical issues. We do not now if start_ts and end_ts just switched places or were recorded incorrectly, so we can't restore them. Let's just exclude the rows from the analysis.

In [ ]:
visits=visits.query('start_ts <= end_ts')
visits.session_length.describe()

Mean session_length is 10 min 43sec, while median is much lower - only 5 min, so the distribution is right skewed. Let's plot a histogtam. First datatypes should be changed and values turned to minutes as numeric.

In [ ]:
visits.session_length.dtypes

Converting timedelta to numeric representation in minutes

In [ ]:
visits['session_length_minutes'] = visits['session_length'].dt.total_seconds() / 60

In [ ]:
visits['session_length_minutes'].describe()

In [ ]:
# Plotting the histogram
visits['session_length_minutes'].hist()

# Adding labels and title
plt.xlabel('Session Length (Minutes)')
plt.ylabel('Frequency')
plt.title('Histogram of Session Length')
plt.show()

There're outliers with length up to 711 min, but 75% of data is below 14 min. Let's 'zoom' the picture.

In [ ]:
visits['session_length_minutes'].hist(bins=100, range = (0,30))
plt.xlabel('Session Length (Minutes)')
plt.ylabel('Frequency')
plt.title('Histogram of Session Length')
plt.show()


In [ ]:
# how many sessions last less than 15 sec
visits.query('session_length_minutes < 0.25')

35794 sessions last less than 0.25 minutes(15 sec!). What is it in %?

In [ ]:
35794/ visits['uid'].count()

The distribution is left skewed, 50% of values are below 5 min. 10% of sessions last less than 15 sec, that can be a signal of some issues with the app(downloading, registration and so on). 

### What's the user retention rate?


**Retention** shows us how many users (in % out of registered) had sessions on a certain day/week/month after first visit. Let's find the first session as min visit date.

In [ ]:
first_ses=visits.groupby(['uid'])['date'].min().reset_index()
first_ses.columns = ['uid', 'first_session']
first_ses.head()

In [ ]:
#now merging to the original dataset
visits=visits.merge(first_ses, how='left',on=['uid'])
visits.head()

For further cohort analysis we will define two parameters:

**cohort**: monthly cohort when the user had first session (what month the user had his first session determines which cohort he belongs to),  
**age**: the difference between any current session and first_session calculated in months.

In [ ]:
visits.info()

In [ ]:
visits['cohort'] = visits['first_session'].dt.to_period('M').dt.to_timestamp()
visits['age'] = ((pd.to_datetime(visits['date']) - pd.to_datetime(visits['first_session'])) / np.timedelta64(1,'M'))\
                                                                        .round().astype('int')
visits.head()

Now cohorts are ready, age is calculated. Lets see how many users were active from certain cohorts on a certain time after registration with pivot table.

In [ ]:
cohorts = visits.pivot_table(index='cohort',
                  columns='age',
                  values='uid',
                  aggfunc='nunique').fillna(0)
cohorts


To calculate **retention** we find a % of those who  are still active from those who registered, that is  we divide columns for all ages by column when age=0

In [ ]:
retention=cohorts.iloc[:,0:].div(cohorts[0], axis=0)
retention

To compare retention rate for different cohorts at different ages heatmap will come in handy.

In [ ]:
# the first line just formats cohort into a str for a nicer output
retention.index=retention.index.astype(str)

sns.heatmap(retention, annot=True, fmt='.1%', linewidths=1, linecolor='grey', vmax=0.1, cbar_kws= {'orientation': 'horizontal'} 
            ).set(title = 'Retention Rate')
plt.show()

The heatmap clearly shows the retention rate(RR) dynamics. All cohorts' RR0(retention rate at the age of 0 months) > RR1 > RR2. But futher there are 2 variants.
*  The first cohort(starting in June) in a month has 7.5 of its users still active, then only 5.4%. At the age of 3 months retention rate rises to 6.2% and remains relatively high for 3 months, than gradually falls to 0.7% at the 12th month. The second(July) cohort has smaller retention rate in a month - only 5.7%(we already know that in august all users show min activity)> then it was smaller for 2 months and again rose to 5.7 in the age of 5 months( which was November the great!). The first two cohorts had the same pattern:  RR0> RR1> RR2 < RR3 < RR4 > RR5 > RR6..., that is after initial decline of retention rate we see a certain rise, then fall again. 
*  All  other cohorts have no rise, their RR is only descending. 

Max retention at 1 month age  - 7.8% - was at September cohort.
February, March, April cohorts have RR less than 5 % from the very beginning.
At the end of the period observed(May) max retention - 4 % - was demonstrated by June cohort(its 12th month) and April cohort(age 1 month - no wonder)! June data are incomplete and won't be analized.

### Conclusion

Metrics about Product were calculated, they show you how users interact with the app. Overall average
**DAU**  is 908 unique users per day, average **WAU** is 5716 unique users per week, average **MAU** is 23228 unique users per month. Their dynamics over the year has much in common.
For nearly half a year(October - March) DAU, WAU and MAU are above average. with the peak value on Nov.24 (BLACK FRIDAY?), before the winter holidays(Cristmass, New Year). Since Aprile DAU, WAU, MAU  are below average,  min values are in august(people are on vacation). There's a sudden drop down around Mar 31 for all metrics. We can also see small regular rises and falls during the week as weekly cycle: people open the app most often on Wednesday, then on Sunday, while min number of visits is on Saturday and Friday. 

**Number of sessions per day** on average is 987 - it is greater than DAU, but the distribution over the year, peaks and falls, week cycles are pretty similar(line graphs almost coincide).

**Number of sessions per user** varies from 1 to 1.22 with the mean 1.08. Unlike other metrics,it is normally distributed, but there are also regular weekly ups and downs, max value is again on Nov 24(Black Friday), min on Mar 31 and in August.

Mean value of **session length** is 10 min 43sec and some users hang on the site up to 700 min, but 50% of sessions last less than 5 min, **10% of sessions last less than 15 sec**, that can be a signal of some issues with the app.

**Cohort analysis**  was made for calculating **retention rate**(RR) as a % of those users who are still active at a certain period of time. Heatmap for RR  for all cohorts at all ages was ploted and RR dynamics was determined.
All cohorts' RR0(retention rate at the age of 0 months) > RR1 > RR2. But then there are 2 scenarios:
-  after initial fall RR can rise again and after 2-3 months begin to fall again(June and July cohorts)
-  no rise, RR is falling every month(all other cohorts).
RR in the first 2 cohorts may rise because in october the general rise in user's activity begins(as we already know about DAU/WAU/MAU and other metrics' dynamics over the year).
-  At the age of 1 month  max retention was 7.8% (September cohort). April, March cohorts activity is the least - RR1  around 4%.
- At the end of the period observed(May) max retention - 4 % - was demonstrated by June cohort(its 12th month) and April cohort(age 1 month - no wonder)! June data are incomplete and won't be analized.


## Sales


### Conversion 
One of the metrics describing sales is Conversion. **Conversion rate** is % of users who made an order(were converted to customers),  **conversion  time** depicts how long does it take from first visit to the purchase in days. So for each user we find the distance between the date of first visit and the date of first order.

In [ ]:
#for each user let's find the time of first order
first_order=orders.groupby(['uid'])['date'].min().reset_index()
first_order.columns = ['uid', 'first_order']
first_order.head()

Earlier we have already found first_session for each user:

In [ ]:
first_ses.head()

Let's merge the info on first session and first order to one user_first table:

In [ ]:
user_first = pd.merge(first_order, first_ses, on = 'uid', how='left')
user_first.head()

Let's calculate average conversion time and plot it

In [ ]:
user_first['conversion']=((user_first['first_order']-user_first['first_session'])/np.timedelta64(1,'D')).astype('int')
user_first['conversion'].describe()

Conversion varies from 0 to 363 days with the mean around 17, while the median is 0 and 75% of users make order within 2 days from the first session.

In [ ]:
fig = px.histogram(user_first, x="conversion",nbins=30)
fig.show()

There are many upper outliers, so let's zoom the picture.

In [ ]:
user_first['conversion'].hist(bins=30, range = (0,30))
plt.xlabel('conversion (days)')
plt.ylabel('Frequency')
plt.title('Histogram of conversion')
plt.show()

So, on average it takes 0 days to convert for a user. But how many users converted at all?

In [ ]:
print(' The overall conversion is {:.1%}'.format(orders['uid'].nunique()/visits['uid'].nunique()))
 

### How many orders do they make during a given period of time?


In [ ]:
#merging the info on first_order to the orders table 
orders=orders.merge(first_order, how='left',on=['uid'])
orders.head()

Let's define cohort on the first order month.

In [ ]:
orders['first_order_month']=orders['first_order'].dt.to_period('M').dt.to_timestamp()
orders.head()

In [ ]:
#let's define cohort size
cohort_sizes = orders.groupby('first_order_month')['uid']. nunique().reset_index()
cohort_sizes.columns=['first_order_month','cohort_size']
cohort_sizes

In [ ]:
#calculating number of purchases for cohort and month
cohort=orders.groupby(['first_order_month','month'])['revenue'].count().reset_index()
cohort.columns=['first_order_month','month','orders']
cohort

In [ ]:
#merge cohort with cohort size
cohort=cohort.merge(cohort_sizes,on=['first_order_month'])
cohort.head()

In [ ]:
#find cohort's age in months
cohort['age_month'] = ((cohort['month'] - cohort['first_order_month']) / np.timedelta64(1,'M')).round()
cohort['orders_per_user']=cohort['orders']/cohort['cohort_size']
cohort.head(20)

We see that every cohort has max number of orders per user(1.1-1.2) in its first month(age 0), then the value becomes many times smaller.

Let's make pivot table for cohorts' orders per user and compare  values for cohorts and months(age).

In [ ]:
cohort_piv=cohort.pivot_table(
    index='first_order_month', 
    columns='age_month', 
    values='orders_per_user', 
    aggfunc='sum'
).cumsum(axis=1)

cohort_piv.round(2).fillna('')

In [ ]:
cohort_piv.index=cohort_piv.index.astype(str)
sns.heatmap(cohort_piv, annot=True, fmt='.2f', linewidths=1, linecolor='grey', cbar_kws= {'orientation': 'horizontal'} 
            ).set(title ='Orders per User(cumulative)')
plt.show()

At the age of 0 and 1 max orders per user is at November cohort - 1.18, 1.28 (no wonder, November is a the happiest month, December is also very active). After that(since age 2) the June cohort becomes the leader and has max cumulative orders per user, but November cohort still has very good results. April and May cohorts are the weakest - cum.orders/user 1.10, 1.09. There'sa also the 13th cohort(June) - someone has made orders on June 1, but it's data is incomplete.

### What is the average purchase size?


In [ ]:
# calculating total check for each user and plotting it
avg_check=orders.groupby(['uid'])['revenue'].sum().reset_index()
avg_check.describe()

User's total check varies a lot - from 0 to 11810, mean revenue is 6.9, but the median is only 3, so the distribution is not normal, but right skewed. There's a big number of 0 revenues:

In [ ]:
orders.query('revenue == 0').head(10)

In [ ]:
orders.query('revenue == 0').revenue.count()

In [ ]:
# share of 0 revenues
51/orders['uid'].count()

In [ ]:
orders.query('revenue == 0')['uid'].value_counts()

Revenues occure on different dates, some users have it just once, but other users have 2-12 such sessions. Maybe the users want to order some tickets and then cansel it, or perchaps it was a child playing with the app.
Share of 0 revenues is not big - 0.1 %, but 0 revenues from orders are impossible, there were no real orders. Let's exclude the rows with 0 revenues fron the analysis.

In [ ]:
orders=orders.query('revenue > 0')
orders.head()

In [ ]:
# calculating total check for each order and plotting it
avg_check['revenue'].hist(bins=50)
plt.xlabel('revenue')
plt.ylabel('Frequency')
plt.title('Histogram of revenue')
plt.show()



In [ ]:
avg_check['revenue'].hist(bins=50, range = (0,100))
plt.xlabel('revenue')
plt.ylabel('Frequency')
plt.title('Histogram of revenue')
plt.show()

In [ ]:
#Cohort analysis on averge check:

avg_cohort=orders.groupby(['first_order_month','month'])['revenue'].mean().reset_index()
avg_cohort['age_month'] = ((avg_cohort['month'] - avg_cohort['first_order_month']) / np.timedelta64(1,'M')).round()
avg_cohort.head()

Now group data in a pivot table with mean revenue value for age(months):

In [ ]:
avg_cohort_piv=avg_cohort.pivot_table(
    index='first_order_month', 
    columns='age_month', 
    values='revenue', 
    aggfunc='mean'
)

avg_cohort_piv.round(2).fillna('')

In [ ]:
avg_cohort_piv.index=avg_cohort_piv.index.astype(str)
sns.heatmap(avg_cohort_piv, annot=True, fmt='.2f', linewidths=1, linecolor='grey', cbar_kws= {'orientation': 'horizontal'} 
            ).set(title ='Revenue')
plt.show()

At the age of 0 July cohort had max revenue(5.29). At the age of 1 month September cohort pulled ahead with  revenue 13.23, and it had absolute max revenue(62.57) at the age of 3 which was in December and was the leader till January.  We can see good results in December cohort, which became the first at its age 2(20.07 in February) and then in March, April, May. Min revenue brought January cohort.

### How much money do they bring? (LTV)


Ltv is the total amount of money the average customer brings to the company by making purchases. It's calculated as gross profit from a customer per period. For this project we can keep margin rate=1, so gp=revenue

In [ ]:
#get the revenue per cohort in each month
ltv_cohort=orders.groupby(['first_order_month','month'])['revenue'].sum().reset_index()
ltv_cohort.head()

In [ ]:
#merge with the cohort size
ltv_cohort=ltv_cohort.merge(cohort_sizes,on=['first_order_month'])
ltv_cohort.head()

In [ ]:

ltv_cohort['age']=((ltv_cohort['month'] - ltv_cohort['first_order_month']) / np.timedelta64(1,'M')).round()
ltv_cohort['ltv']=ltv_cohort['revenue']/ltv_cohort['cohort_size']
ltv_cohort.head()

In [ ]:
# Calculate LTV with cumsum in pivot table
ltv_cohort_piv=ltv_cohort.pivot_table(
    index='first_order_month', 
    columns='age', 
    values='ltv', 
    aggfunc='sum'
).cumsum(axis=1)
ltv_cohort_piv

Let's visualize it with heatmap.

In [ ]:
ltv_cohort_piv.index=ltv_cohort_piv.index.astype(str)
sns.heatmap(ltv_cohort_piv, annot=True, fmt='.2f', linewidths=1, linecolor='grey', cbar_kws= {'orientation': 'horizontal'} 
            ).set(title ='LTV')
plt.show()

Cumulative LTV naturally grows in all rows. Again September cohort is the leader - it has max LTV since age 1(Oktober) throgh most profitable Nov,Dec till May, reaching the value of 13.44. If we compare diagonal values, June cohort starts out - it has max of all cohorts LTV in Oct(age 4 - 7.62) and Nov, then it is the second after September cohort. The least profitable is February cohort with min LTV(4.59).

### Conclusion

Sales metrics were calculated.

The overall **conversion**  is 16.0%, and 50% of users make order  and become customers on the same day as they are registered so the median of **conversion time** is 0.  75% of users make order within 2 days from the first session, but the mean conversion time is much higher (around 17), some users wait up to 363 days before ordering.

Number of **orders per user** was calculated for month cohorts. Every cohort has max number of orders per user in its first month(age 0), then the value becomes many times smaller, but cumulative sum is gradually rising. At the age of 0 and 1 max cum orders per user is at November cohort - 1.18, 1.28 (no wonder, November is a the happiest month due to Black Friday, December is also very active). After that(since age 2) the June cohort becomes the leader and has max cumulative orders per user. April and May cohorts are the weakest - cum.orders/user 1.10, 1.09.

User's total check for **revenue** is 6.9(the mean), but the median is only 3, so the distribution is  right skewed with outliers up to 11810.  0 revenues(0,1%) were excluded from cohort analysis.
At the age of 0 July cohort had max revenue(5.29).  At the age of 1 month September cohort pulled ahead with  revenue 13.23, and it had absolute max revenue(62.57) at the age of 3 which was in December and was the leader till January.  We can see good results in December cohort, which became the first at its age 2(20.07 in February) and then in March, April, May. Min revenue brought January cohort.

Cumulative **LTV** naturally grows in all rows. Again September cohort  brought to the company max money - it has max LTV since age 1(Oktober) throgh most profitable Nov,Dec till May, reaching the value of 13.44. The second is June cohort (cum LTV 11.88). The weakest is February cohort with min LTV(4.59).


##  Marketing


### How much money was spent? 

#### Cost: overall and per source

Here we deal with costs dataset.

In [ ]:
#sorting cost for futher visualization
costs=costs.sort_values(by=['dt','source_id'])
costs.head()

In [ ]:
# find total marketing
print('Total marketing cost is {}'.format(costs['costs'].sum()))

So, **total amount spent on marketing**  by the company is $329131.62. Let's find its distribution per source.

In [ ]:
marketing_source=costs.groupby(['source_id'])['costs'].sum()
marketing_source

In [ ]:
# Calculate the percentage of each source's cost over the total
percentage = (marketing_source / marketing_source.sum() * 100).round(1)
result = pd.DataFrame({'total_costs': marketing_source, 'percentage': percentage})
result = result.sort_values('total_costs', ascending=False).reset_index()
result

In [ ]:
# Plot the barplot for total costs per source
result.plot(kind='bar', x='source_id', y='total_costs', color='blue')
plt.xlabel('Source')
plt.ylabel('Total Costs')
plt.title('Marketing: Total Costs per Source')
plt.show()

**So**,  most of the money was spent on source 3(43% of costs, 141321.63 costs), the smallest amount was spent on sources 9,10 (less than 2%).

#### Costs over time

Let's see how costs change  over time - each day, month, over the week.

In [ ]:
import plotly.express as px
fig = px.line(costs, x="dt", y="costs", color='source_id')
fig.show()

Lines for all sources are shown on the graph. Source 3 has max values and max fluctuation range, soeces 9,10 have min values.The highest peak for all source is on Nov 24 - as expected, min costs were spent on Mar 31(end of quarter?) and in august(people on holidays). There're also some minor peaks for source1: Okt 6, Okt 27, Nov 3, Dec 11, Dec 28, Jan3, Mar 7, May 31, June 8. Close to them but not always matching are peaks for source 4: Dec 5,14, Jan 16,22, Feb 13, Mar 10,12, June 9, Jul 14. For some reason there's no special activity of source 4 around Nov 24 -  on the contrary, its costs are pretty smalll on Nov 24.
Source 5  has max costs on Nov 24 oand just one more peak on May 31.

Let's unite all the sources and see the whole marketing costs distribution over the year.

In [ ]:
total_date=costs.groupby(['dt'])['costs'].sum().reset_index()
total_date

In [ ]:
fig = px.line(total_date,x="dt", y="costs",title='Costs per day')

fig.add_hline(y=total_date['costs'].mean(),line_dash="dash", line_color="purple", annotation_text="average costs",
             annotation_position="top left")
fig.show()

This graph is very similar to DAU/WAU/MAU/number of sessions per day lineplots with the same peak on Nov 24  and min costs spent in August and sudden fall on Mar 31.

Let's compare with revenue distribution over the year.

In [ ]:
revenue_date=orders.groupby(['date'])['revenue'].sum().reset_index()
fig = px.line(revenue_date,x="date", y="revenue",title='Revenue per day')

fig.add_hline(y=revenue_date['revenue'].mean(),line_dash="dash", line_color="purple", annotation_text="average costs",
             annotation_position="top left")
fig.show()

There's less correlation than with DAU/...metrics. The highest peak is not on Nov 24, but on Dec 10 (related with max activity of source 5 supported by other sources? people buy more tickets before winter holidays?) and May 31. Revenue in January falls below average(users have already spent all the money) till Jan 26, Feb 1 and stay above average till the end of March. Min revenue on Mar 31 and in august correlates with min marketing costs. 

Let's find monthly marketing costs and compare them with revenue.

In [ ]:
marketing_month=costs.groupby(['month'])['costs'].sum().reset_index()
marketing_month.plot(kind='bar', x='month', y='costs', color='blue')
plt.xlabel('Month')
plt.ylabel('Total Costs')
plt.title('Marketing: Total Costs per Month')
plt.show()

So, the largest costs are spent on marketing in December, November, October, the smallest costs are is August.

In Oct, Nov and Dec marketing costs gradually grow, that correlates with growing revenues and max revenue in December. In January costs remain relatively high, though  revenue falls below average. Costs decrease from jan to March, while DAU/MAU doesn't change much and revenue per day grows towards March. Costs in June and July are the equal, but revenue and users activity rises significantly around July 17. Mayby marketing costs should support natural user's activity in February, March and July.

There're also weekly fluctuations in costs, as in user activity metrics. 

In [ ]:
costs['dow'] = costs['dt'].dt.dayofweek
total_dow=costs.groupby(['dow'])['costs'].sum()
total_dow

There's definetly a difference in costs depending on day of the week. Let's plot barplot(1 stays for Monday).

In [ ]:
total_dow.plot(kind='bar', x='dow', y='costs', color='blue')
plt.xlabel('Day of Week')
plt.ylabel('Total Costs')
plt.title('Marketing: Total Costs per Day of Week')
plt.show()

So, the largest amount is spent on marketing on Wednesday, then Thursday and Sunday, min costs are on Saturday. We already now, that users open the app most often on Wednesday, then on Sunday, while min number of visits is on Saturday and Friday. Maybe the main target days should be  Wednesday and Sunday(not Thursday) to attract more customers.


### How much did customer acquisition from each of the sources cost?


#### CAC

In [ ]:
customers_month=orders.groupby(['first_order_month'])['uid'].nunique().reset_index()
customers_month.columns=['month','customers']
customers_month.head()

In [ ]:
CAC_month=marketing_month.merge(customers_month,how='left',on=['month'])
CAC_month.head()

In [ ]:

CAC_month['CAC']=CAC_month['costs']/CAC_month['customers']
CAC_month.head()

In [ ]:
fig = px.line(CAC_month, x="month", y="CAC", title='CAC')
fig.show()

Max CAC is in August(10,8), when user's activity is the least,then it goes down till October(8.4), when users become more active. In November attracting a customer costs more than in December(users are active without special push) and March, but less than in January and April(users don't want to buy). Min CAC is in May(7,4)

#### CAC per source

Not all users have one source, let's define the first that they came from as the main source.

In [ ]:
first_source=visits.sort_values('date').groupby('uid').first()['source_id'].reset_index()
first_source.columns=['uid','first_source']
first_source.head()

Merge this data back to orders becaue CAC calculations are based on buyers

In [ ]:
orders=orders.merge(first_source,on=['uid'],how='left')
orders.head()

Let's see how CAC was changing for each source across time 

In [ ]:

marketing_month_source=costs.groupby(['month','source_id'])['costs'].sum().reset_index()
marketing_month_source.head()

In [ ]:
customer_month_source=orders.groupby(['first_order_month','first_source'])['uid'].nunique().reset_index()
customer_month_source.columns=['month','source_id','customers']
customer_month_source.head()

In [ ]:
#merging tables with data for cac
CAC_month_source=marketing_month_source.merge(customer_month_source,how='left',on=['month','source_id'])
CAC_month_source.head()

In [ ]:
#actual cac calculations
CAC_month_source['CAC']=CAC_month_source['costs']/CAC_month_source['customers']
CAC_month_source.head()

In [ ]:
#plotting cac dynamics

fig = px.line(CAC_month_source, x="month", y='CAC',color='source_id',title='CAC')
fig.show()

All seven sources are shown, their lines differ much. Sources 3,2,4 have max CAC in August,
source 5 - in November, source 1 - in January, source 10 - in December, source 5 - in  April. All sources lines fall in May.

Lines for souces 3,2  are much higher than others. Let's calculate overall average CAC per source  and compare number of customers and costs.

In [ ]:
source_CAC=CAC_month_source.groupby(['source_id'])[['CAC','customers', 'costs']].agg({'customers': 'sum', 'costs': 'sum', 'CAC': 'mean'}).sort_values(by='CAC', ascending=False)
source_CAC

So, sources 3 and 2 spend much more on attracting a customer than other sources and they bring many customers. But these sources are not so effective as sources 5, 4 ,1: source 4 brings almost as many customers as source 3, but each customers price is 2 times lower, source 1 brings the same number of customers as source 2 at 2 times lower cost. Maybe there's a possibility to reduce costs on less effective sources in favor of most effective sources.

### How worthwhile where the investments? (ROI)


#### ROI per cohort.

In genereal, ROI=LTV/CAC

We already have calculations on CAC per month and we have info on ltv in ltv_cohort. So let's merge and work it out!

In [ ]:
CAC_month_ROI=CAC_month[['month','CAC']]
CAC_month_ROI.columns=['first_order_month','CAC']
ROI=ltv_cohort.merge(CAC_month_ROI,on=['first_order_month'],how='left')
ROI.head()

In [ ]:
ROI['ROI']=ROI['ltv']/ROI['CAC']
roi_piv = ROI.pivot_table(
    index='first_order_month', columns='age', values='ROI', aggfunc='mean'
).cumsum(axis=1).round(2)

In [ ]:
roi_piv.index=roi_piv.index.astype(str)
sns.heatmap(roi_piv, annot=True, fmt='.2f', linewidths=1, linecolor='grey', cbar_kws= {'orientation': 'horizontal'} 
            ).set(title ='ROI per cohort')
plt.show()

Again we see the best results for September cohort reaching ROI 1.42  and June cohort ROI 1.33. Only these cohorts'  ROI is above 1, so campaign's gross profit exceeds expenses. Other cohorts have ROI smaller than 1, so for them ad campaing doesn't pay off. 

#### ROI per source 

In [ ]:
ltv_source=orders.groupby(['first_source'])[['uid','revenue']].agg({'uid':'nunique','revenue':'sum'}).reset_index()
ltv_source.columns=['source_id','customers','revenue']
ltv_source['ltv']=ltv_source['revenue']/ltv_source['customers']
ltv_source

In [ ]:
marketing_source=marketing_source.reset_index()
marketing_source

In [ ]:
roi_source=marketing_source.merge(ltv_source,on=['source_id'])
roi_source['cac']=(roi_source['costs']/roi_source['customers']). round(2)
roi_source['romi']=(roi_source['ltv']/roi_source['cac']).round(2)
roi_source

In [ ]:
fig = px.bar(roi_source, x='source_id', y='romi')
fig.update_xaxes(type='category')
fig.show()

So, source 1 has max ROMI(1.8), sources 2 and 5 also have ROMI > 1 ,so they are profitable. Source 9 has ROMY equal to 1. Source 3 has min ROMI 0.37, sources 10 - 0.78, source 4 - 0.9. Probably sources 3 and 10 should be cut out, while the flow from sources 1,2,5 should be increased.

### Conclusion
Marketing metrics were calculated and analized.

**total costs** spent on marketing is $329131.62. Max **share per source** - 43% of all costs - is from source 3, min costs -  less than 2% - from sources 9,10.
**Over time**: the highest peak for almost all sources is on Nov 24 - as expected, min costs were spent on Mar 31(end of quarter?) and in august(people on holidays). There're also some minor peaks for sources 1, 4, 5 which often(not always) match. For some reason there's no special activity from source 4 around Nov 24 -  on the contrary, its costs are pretty small on Nov 24.
The distribution of total costs over time is compered with:
-  **DAU/WAU/MAU/number of sessions per day** lineplots -  very similar to  total costs, with the same peak on Nov 24  and min costs spent in August and sudden fall on Mar 31. 
- **revenue** distribution over time - less correlation found. The highest revenue peak is not on Nov 24, but on Dec 10 (related with max activity of source 5 supported by other sources? people buy more expensive goods befor the main holidays) and May 31. Revenue in January falls below average(users have spent all the money on expensive presents) till Jan 26, Feb 1 and stay above average till the end of March. Min revenue on Mar 31 and in august correlates with min marketing costs.

**Marketing costs per month**: max costs are spent on marketing in December, November, October, the smallest costs are is August. In Oct, Nov and Dec marketing costs gradually grow, that correlates with growing revenues and max revenue in December. In January costs remain relatively high, though revenue falls below average. Costs decrease from jan to March, while DAU/MAU doesn't change much and revenue per day grows towards March. Costs in june and July are the same, but revenue and users activity rises significantly around July 17. Mayby marketing costs should support more natural user's activity in February, March and July?

**Marketing costs by day of the week**: regular weakly cycles are seen on the graph for daily costs like on DAU and revenue graphs. Max marketing costs are spent on Wednesday, then Thursday and Sunday, min costs are on Saturday. As we have already seen users open the app most often on Wednesday and Sunday, while min number of visits is on Saturday and Friday. Maybe the main target days should be  Wednesday and Sunday(not Thursday) to attract more customers.

**CAC over time**: Max CAC is in August(10,8), when user's activity is the least, then it goes down till October(8.4), when users become more active. In November attracting a customer costs more than in December(users are active without special push) and March, but less than in January and April(users don't want to buy). Min CAC is in May(7,4).

**CAC per source**:
Sources 3 and 2 spend much more on attracting a customer than other sources and they bring many customers. But these sources are not so effective as sources 5,4,1: Source 4 brings almost as many customers as source 3, but each customers price is 2 times lower, source 1 brings the same number of customers as source 2 at 2 times lower cost.
Maybe there's a possibility to reduce costs on less effective sources in favor of most effective sources.

**ROI** in cohort analysis
The best results for September cohort reaching ROI 1.42  and June cohort ROI 1.33. Only these cohorts'  ROI is above 1, so campaign's gross profit exceeds expenses. Other cohorts have ROI smaller than 1, so for them ad campaing doesn't pay off. 

**ROI per source**
source 1 has max ROMI(1.8), sources 2 and 5 also have ROMI > 1 ,so they are profitable. Source 9 has ROMY equal to 1. Source 3 has min ROMI 0.37, sources 10 - 0.78, source 4 - 0.9. Probably sources 3 and 10 should be cut out, while the flow from sources 1,2,5 should be increased.


# Step 3. Write a conclusion: advise marketing experts how much money to invest and where.


Our conclusions about user's activity, sales and marketing metrics help to find problems with the app and recommend some changes in marketing organization.

**Cohorts** max activity was shown by June and September cohorts - max retention, max cumulatibe orders per user, revenue, LTV , ROI, while other cohorts were only effective in some periods by some metrics( eg.december cohort had max revenue in February, March, April). Theese profitable cohorts are supported not only by natural factors such as rises in users activity over the year, but also by special marketing efforts. Marketing department should use this positive experience to support other cohorts. 

**Sources** Max share of marketing costs(43% of all costs) is related with source 3, min costs (less than 2%) with  sources 9,10.  Sources 4 ,5 , 1 get only 18.6%, 15.7% , 6,3% respectively (summed less than source 3), but they are much more effective! Comparing CAC per source shows that sources 3 and 2 have max CAC(14.2) 12.3), so source 4 brings almost as many customers as source 3 with  2 times lower CAC(6.6), source 5 brings 2 times more customers than source 2 with 2 times lower CAC? source 1 brings the same number of customers as source 2 at 2 times lower cost. Analizing return on marketing investments we find that only sources 1,2,5 have ROMI >1 ( max 1.8 - source 1) and make profit, source 9 has ROMY equal to 1, other sources don't pay off. Source 3 has min ROMI 0.37, sources 10 - 0.78. Probably sources 3 and 10 should be cut out, while the flow from sources 1,2,5 should be increased to reduce costs on less effective sources in favor of most effective sources.

**Over time** 
In general marketing costs are distributed over year in correlation with user's activity, which is above average since October till April and has peaks on  Nov 24, May 31 and is minimum in August when people are on vacation.  
- The majority of sources spend max costs on Nov 24, which is Black Friday, and users also demonstrate max activity on that day. But the highest revenue peak is not on Nov 24, but on Dec 10, which correlates with max costs spend on source 5(maybe people by more tickets before winter hilidays). We may suppose that if other sources concentrate not only on Nov 24, but also on December sales, they will get more revenue. 
- In January revenue falls below average (users have already spent all their money), and rises again by Jan 26 - Feb 1, then stays above average till the end of March. But in January marketing costs remain relatively high, then costs decrease from January to March(while DAU/MAU doesn't change much and revenue per day grows towards March). Marketing costs in June and July are equal, but revenue and users activity rises significantly around July 17. We suppose that costs destribution should take into account not only user's activity, but also revenue distribution over the year, thus costs would boost natural user's activity and bigger revenue in February, March and July.
- Marketing costs per month: max costs are spent on marketing in December, November, October, the smallest costs are is August. In Oct, Nov and Dec marketing costs gradually grow, that correlates with growing revenues and max revenue in December. From analizing CAC we know that max CAC is in August(10,8), when user's activity is the least, then it goes down till October(8.4), when users become more active. In November attracting a customer costs more than in December(users are active without special push) and March, but less than in January and April(users don't want to buy). Min CAC is in May(7,4). This picture gives one more reason to increase marceting costs in December, March and May and decrease in August.
- Day of week: Maximum user activity occurs on Wednesday and Sunday( max DAU, number of sessions per user, session length), but now max marketing costs are spent on Wednnesday and Thursday. It would be better to focus on Sunday instead of Thursday, so the main target days would be Wednesday and Sunday(not Thursday) to attract more customers.


**Technical issues**
- 10% of sessions last less than 15 sec -  that can be a signal of some issues with the app (starting page loading, registration form etc) Tecnical support should identify and fix these issues.
- There was a drop down in all product metrics as well as sales and marketing metrics on March 31. Check if there were any problems with the app which could explain the issue.
 

